In [5]:
%pip install --upgrade --quiet langchain "unstructured[gcs,all-docs]" langchain openai langchain-openai unstructured[pdf] langchain-community langchain-core langchain_chroma langchainhub pyautogen python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the API key from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")

# Ensure the API key is set
if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable not found")

from langchain_openai import ChatOpenAI

# Initialize the LLM with the API key
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=openai_api_key)

import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import TokenTextSplitter

# Read the PDF document
loader = UnstructuredPDFLoader("E:\\Code\\GraphRag\\dataset\\sanei.pdf")
docs = loader.load()

# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(docs[:3])

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
def rag_pdf(text : str) -> str:
    return rag_chain.invoke(text)

In [58]:
import json
class AppendResult(BaseModel):
    status: str
    message: str
def append_to_jsonl( data :DatasetRow)  -> AppendResult :
    """
    Appends a JSON object to a JSONL file.
    
    Parameters:
    - file_path: str, path to the JSONL file
    - json_object: dict, JSON object to append
    """
    try:
        with open("dataset.jsonl", 'a') as f:
            f.write(json.dumps(data.dict()) + '\n')
        return AppendResult(status="success", message="Dataset row appended successfully.")
    except Exception as e:
        return AppendResult(status="error", message=str(e))

In [83]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel
from autogen import ConversableAgent, GroupChat, GroupChatManager, register_function
from autogen.coding import LocalCommandLineCodeExecutor

# Load environment variables from .env file
load_dotenv()

# Define entry value
entry = 100

# Define DatasetRow class
class DatasetRow(BaseModel):
    question: str
    answer: str

# Define LocalCommandLineCodeExecutor
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir="dataset/",  # Use the temporary directory to store the code files.
)

# Define assistant agent
assistant = ConversableAgent(
    name="Assistant",
    system_message=f"You are a helpful AI assistant in neuroscience. "
    "You can help to build dataset for eeg application with {entry} entries. "
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

# Define code executor agent
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="NEVER",  # Always take human input for this agent for safety.
)

# Define user proxy agent
user_proxy = ConversableAgent(
    name="User",
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
    code_execution_config={"executor": executor},  # Use the local command line code executor.
)

# Placeholder function definitions for rag_pdf and append_to_jsonl
def rag_pdf(pdf_path: str) -> str:
    # Placeholder function for extracting data from PDF
    return "Extracted data from PDF"

def append_to_jsonl(json_object: dict, file_path: str):
    # Placeholder function for appending data to a JSONL file
    with open(file_path, 'a') as f:
        f.write(f"{json.dumps(json_object)}\n")

# Register functions
register_function(
    rag_pdf,
    caller=assistant,  # The assistant agent can suggest calls to the function.
    executor=user_proxy,  # The user proxy agent can execute the function calls.
    name="rag_pdf",  # By default, the function name is used as the tool name.
    description="Used to retrieve data in a PDF",  # A description of the tool.
)

register_function(
    append_to_jsonl,
    caller=assistant,  # The assistant agent can suggest calls to the function.
    executor=user_proxy,  # The user proxy agent can execute the function calls.
    name="append_to_jsonl",  # By default, the function name is used as the tool name.
    description="Used to append a JSON object to a dataset file",  # A description of the tool.
)

# Create group chat and manager
groupchat = GroupChat(agents=[user_proxy, assistant], messages=[], admin_name=assistant.name)
manager = GroupChatManager(groupchat=groupchat, llm_config={"config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]})

# Start the automated chat


In [84]:
res = user_proxy.initiate_chat(assistant, message= f"You must create {entry} entries for datasets that will be used for fine-tuning an AI model on EEG signal analysis based on the pdf you read. Each entry should include the following fields: 'question', 'answer'. The 'question' should be a scientifique one (algorithm, concept, hypothesis, conslusion,) you must be innovative, the 'answer' should be a good quality technical data (formula, algorithm, mathematic, concept, hypothesis...) retrieve in the datasource. Everytime append the answer in a jsonl file with the following format {{question : the question, answer : the answer}}. ", clear_history=False)


User (to Assistant):

You must create 100 entries for datasets that will be used for fine-tuning an AI model on EEG signal analysis based on the pdf you read. Each entry should include the following fields: 'question', 'answer'. The 'question' should be a scientifique one (algorithm, concept, hypothesis, conslusion,) you must be innovative, the 'answer' should be a good quality technical data (formula, algorithm, mathematic, concept, hypothesis...) retrieve in the datasource. Everytime append the answer in a jsonl file with the following format {question : the question, answer : the answer}. 

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_HVPlD3c4DsJfBhtSmYb2la6s): rag_pdf *****
Arguments: 
{"text":"EEG signals are commonly analyzed using a variety of signal processing techniques. These include time-domain analysis, frequency-domain analysis, and time-frequency-domain 

[runtime logging] log_function_use: autogen logger is None


User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_HVPlD3c4DsJfBhtSmYb2la6s) *****
EEG signals are commonly analyzed using time-domain, frequency-domain, and time-frequency-domain analysis techniques. Machine learning techniques are widely used for classification and prediction purposes in EEG signal analysis. Algorithms have been developed for visualizing brain activity and separating desired sources from multisensor EEGs.
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_NhcRnfW5B0OlzpAfEUluvS8m): append_to_jsonl *****
Arguments: 
{"data": {"question": "What are some common signal processing techniques for EEG signal analysis?", "answer": "EEG signals are commonly analyzed using time-domain, frequency-domain, and time-frequency-domain analysis techniques."}}
**********

[runtime logging] log_function_use: autogen logger is None



>>>>>>>> EXECUTING FUNCTION append_to_jsonl...


...


>>>>>>>> EXECUTING FUNCTION append_to_jsonl...


[runtime logging] log_function_use: autogen logger is None


User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_HLoWxc5dzrkYZ4eolTEmhfms) *****
{"status":"success","message":"Dataset row appended successfully."}
**********************************************************************

--------------------------------------------------------------------------------
User (to Assistant):

***** Response from calling tool (call_GdmqAnJmFSKuHSaFVcwpYIYD) *****
{"status":"success","message":"Dataset row appended successfully."}
**********************************************************************

--------------------------------------------------------------------------------
User (to Assistant):

***** Response from calling tool (call_ZLDnnr0XkJunfJYY2GSOSu17) *****
{"status":"success","message":"Dataset row appended successfully."}
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assist

[runtime logging] log_function_use: autogen logger is None


User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_POOEMnjNVatX6m7S23Xod4Ys) *****
{"status":"success","message":"Dataset row appended successfully."}
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
User (to Assistant):

***** Response from calling tool (call_POOEMnjNVatX6m7S23Xod4Ys) *****
{"status":"success","message":"Dataset row appended successfully."}
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

We have successfully created 100 entries for the dataset. The task is now complete.

TERMINATE

--------------------------------------------------------------------------------
